經過第一輪嘅標註同處理之後，我哋要做下面步驟：

1. （完成）將下面兩個文件全部擺落`hand_extracted.csv`
  1. `output/jyutping_useful.csv`
  1. `re_output/jyutping_useful.csv`
1. （完成）將 `re_output/no_jyutping_useful.csv` 加粵拼並揾人人工檢查之後加入 `hand_extracted.csv`
1. （完成） `error_words.csv` 放入 `fixed.csv`檢查修正後加入總表
1. （完成）將所有 `jyutping_junk.csv` 嘅詞，人手篩選有用嘅加入總表
1. （完成）將`round_1_annotated/re_output/no_jyutping_junk.csv`入面嘅詞條人工篩選有用嘅放落 `hand_extracted.csv`

1. 將`hand_extracted.csv` 嘅詞修正常見錯別字同語氣詞用字，同埋官話詞「吃」


# 第二輪整理

跟住將總表入面嘅所有詞彙進行下面嘅處理工作：

1. （暫擱）整理出所有同音詞，然後討論規範寫法，刪除唔標準嘅寫法
1. 將所有詞條轉成 OpenCC 字形，去重排序

跟住就**發佈初版**，然後開始下一輪嘅人工處理：


In [16]:
# Release
import pandas as pd
import opencc
converter = opencc.OpenCC('hk2t.json')

extracted = pd.read_csv('./hand_extracted.csv')
hbl= pd.read_csv('./hbl_filtered.csv')
wordshk = pd.read_csv('./wordshk_filtered.csv')
data = pd.concat([extracted, hbl, wordshk], ignore_index=True)


data['word'] = data['word'].apply(converter.convert)
data.drop_duplicates(inplace=True)
data.sort_values(by=['jyutping', 'word'], inplace=True)

data.to_csv("released.csv", index=False)

In [ ]:


common_char = r'(佢|我|你|唔|啊|啦|喇|㗎|咗|係|喺|嘅|嘢|乜|咁|冇|噉|咩|啲|諗|仲)'
condition = data['word'].str.contains(common_char)
phrase = data[condition]
remain = data[~condition]
phrase.to_csv('./release_phrase.csv', index=False)

long = remain[remain['word'].str.len() > 4]
remain = remain[remain['word'].str.len() <= 4]
long.to_csv('./release_long.csv', index=False)

multigraph = {}
for i, row in data.iterrows():
    if row['jyutping'] not in multigraph:
        multigraph[row['jyutping']] = []
    multigraph[row['jyutping']].append(row['word'])

multi_list = []
for jyutping, words in multigraph.items():
    if len(words) > 1:
        multi_list.append([jyutping, words])
pd.DataFrame(multi_list).to_csv('homophone.csv', index=False)

